In [ ]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az

az.style.use("arviz-darkgrid")

In [ ]:

# Set seed for reproducibility
np.random.seed(0)

# Parameters for the simulation
n_firms = 100  # Number of firms
n_years = 50  # Number of time periods
n_obs = n_firms * n_years  # Total number of observations

alpha_capital = 0.3  # Elasticity of output with respect to capital
alpha_labor = 0.6  # Elasticity of output with respect to labor
TFP_mean = 1.0  # Average Total Factor Productivity (TFP)
TFP_std = 0.1  # Standard deviation of TFP (random noise)
error_std = 0.05  # Standard deviation of random error

# Step 1: Generate firm and time period identifiers
firm_ids = np.repeat(range(n_firms), n_years)
time_ids = np.tile(range(n_years), n_firms)

# Step 2: Simulate capital (K), labor (L), and TFP (A)
capital = np.random.uniform(10, 50, n_obs)  # Capital (K)
labor = np.random.uniform(5, 25, n_obs)  # Labor (L)
TFP = np.random.normal(TFP_mean, TFP_std, n_obs)  # TFP (A)

# Step 3: Generate the output (Y) using the Cobb-Douglas production function
output = TFP * (capital ** alpha_capital) * (labor ** alpha_labor)

# Step 4: Add random error to the output
error = np.random.normal(0, error_std, n_obs)  # Random noise
output += error  # Final output

# Step 5: Construct the DataFrame
df = pd.DataFrame({
    'firm': firm_ids,
    'year': time_ids,
    'output': output,
    'capital': capital,
    'labor': labor,
    'TFP': TFP
})

# Step 6: Set the panel data index
df = df.set_index(['firm', 'year'])

# Step 7: Log-transform the variables (for the Cobb-Douglas function)
df['log_output'] = np.log(df['output'])
df['log_capital'] = np.log(df['capital'])
df['log_labor'] = np.log(df['labor'])

# Display the first few rows of the simulated data
# print(df.head())

# Step 8: Fit a fixed effects model
model = PanelOLS.from_formula('log_output ~ log_capital + log_labor + EntityEffects', data=df)
fe_results = model.fit()

# Print the results of the fixed effects model
print(fe_results)


In [ ]:
# Step 8: Fit a Bayesian Model using PyMC
with pm.Model() as model:
    # Priors for the elasticities (alpha_capital, alpha_labor)
    alpha_capital = pm.Normal('alpha_capital', mu=0.3, sigma=0.1)
    alpha_labor = pm.Normal('alpha_labor', mu=0.6, sigma=0.1)

    # Priors for firm-specific fixed effects (entity effects)
    firm_effects = pm.Normal('firm_effects', mu=0, sigma=1, shape=n_firms)

    # Model error term (for the residuals)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Convert firm indices to numpy array for indexing
    firm_indices = df.index.get_level_values('firm').to_numpy()

    # Linear model: log_output = alpha_capital * log_capital + alpha_labor * log_labor + firm_effects + error
    log_output_est = (alpha_capital * df['log_capital'] +
                      alpha_labor * df['log_labor'] +
                      firm_effects[firm_indices] )  # Correctly index firm_effects

    # Likelihood (Gaussian)
    likelihood = pm.Normal('likelihood', mu=log_output_est, sigma=sigma, observed=df['log_output'])

    # Step 9: Inference (MCMC)
    trace = pm.sample(2000, return_inferencedata=True)

# Step 10: Summarize the results
az.summary(trace, var_names=['alpha_capital', 'alpha_labor', 'firm_effects'])

# Plot the results
az.plot_trace(trace, var_names=['alpha_capital', 'alpha_labor'])
plt.show()

# Optionally, check posterior predictive checks (PPC)
pm.plot_posterior(trace)
plt.show()

In [ ]:
az.summary(trace)